In [ ]:
!pip install tensorflow --upgrade
!pip install keras --upgrade
!pip install --upgrade tensorflow-metadata


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

# Loading the training dataset
url = 'https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/hau/train.tsv'
df = pd.read_csv(url, sep='\t')

# Extract the reviews and labels from the training dataset
reviews = df['tweet'].values
labels = df['label'].values

# Split the training dataset into training and validation sets
train_reviews, val_reviews, train_labels, val_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Tokenizing the training reviews
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_reviews)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_reviews)
max_len = max(len(x) for x in sequences)
train_data = pad_sequences(sequences, maxlen=max_len)

# Tokenize the validation reviews
val_sequences = tokenizer.texts_to_sequences(val_reviews)
val_data = pad_sequences(val_sequences, maxlen=max_len)

# One-hot encode the labels for training and validation sets
train_labels = np.array(pd.get_dummies(train_labels))
val_labels = np.array(pd.get_dummies(val_labels))


# Build the model
model = Sequential()
model.add(Embedding(5000, 32, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=5, batch_size=64)

# Load the test dataset
test_url = 'https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/data/hau/test.tsv'
test_df = pd.read_csv(test_url, sep='\t')

# Tokenize the test reviews
test_sequences = tokenizer.texts_to_sequences(test_df['tweet'].values)
test_data = pad_sequences(test_sequences, maxlen=max_len)

# Predict the labels for the test reviews
predictions = model.predict(test_data)

# Maping the predicted labels back to the original label names
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = np.vectorize({0: 'negative', 1: 'neutral', 2: 'positive'}.get)(predicted_labels)

# Save the predicted labels to a text file
with open('/dev/predicted_labels.txt', 'w') as file:
    file.write('\n'.join(predicted_labels))



Epoch 1/5
178/178 [==============================] - 4s 14ms/step - loss: 0.9302 - accuracy: 0.5390 - val_loss: 0.7733 - val_accuracy: 0.6201
Epoch 2/5
178/178 [==============================] - 3s 14ms/step - loss: 0.6560 - accuracy: 0.7132 - val_loss: 0.6713 - val_accuracy: 0.6991
Epoch 3/5
178/178 [==============================] - 3s 18ms/step - loss: 0.4514 - accuracy: 0.8245 - val_loss: 0.6862 - val_accuracy: 0.7005
Epoch 4/5
178/178 [==============================] - 2s 13ms/step - loss: 0.3114 - accuracy: 0.8885 - val_loss: 0.8245 - val_accuracy: 0.6917
Epoch 5/5
166/166 [==============================] - 1s 3ms/step
